In [1]:
import numpy as np
import plotly as px
import pandas as pd
import plotly.express as px
from copy import deepcopy
from power import *
from power_flow import *
from power.systems import *
from optimal_power_flow.studies.ac_econ_dispatch import ACEconDispatch
from optimal_power_flow.studies.ac_min_deviation import ACMinDeviation
from optimal_power_flow.studies.dc_iterative_loss import OPFIterativeLoss
from optimal_power_flow.core.ac_physics import OPFAC
from optimal_power_flow.core.dc_physics import OPFDC
from trabalhos_transmissao.utils.load_scen import apply_load_scen
from trabalhos_transmissao.utils.wnd_scen import apply_wnd_scen

In [2]:
net = B3EOLCharged()
sol = ACEconDispatch(net)
sol.solve()

{'Resumo':     Total_Cost  Total_P_Shed_MW  Total_Q_Virtual_MVAr   Status
 0  1418.467485         3.145987              0.000138  Optimal,
 'Thermal_Generation':                  Name    Bus      P_MW    Q_MVAr     S_MVA  P_Max  Q_Min  \
 0  ThermalGenerator_2  Bus_2  8.003634  0.933783  8.057922   30.0    0.0   
 
    Q_Max  Cost  
 0   15.0  2000  ,
 'Wind_Generation':               Name    Bus     P_MW        Q_MVAr    S_MVA  Available  \
 0  WindGenerator_1  Bus_1  8.96578  9.067277e-34  8.96578       30.0   
 
    Curtailment  
 0     21.03422  ,
 'Battery': Empty DataFrame
 Columns: []
 Index: [],
 'Load_Shed':      Load    Bus  P_Load_MW  P_Shed_MW  Q_Virtual_MVAr  Cost_P
 0  Load_1  Bus_3       20.0   3.145987        0.000138   40000,
 'Bus':      Bus      V_pu  Angle_deg  LMP_P  LMP_Q
 0  Bus_1  1.050000   0.000000    0.0    0.0
 1  Bus_2  1.049013  -1.044474    0.0    0.0
 2  Bus_3  1.041414  -3.646218    0.0    0.0,
 'Line':      Line   From     To     P_From    Q_From    P_

In [3]:
def check_violations(net: Network, tol=1e-4):
    violations = {
        "v_mag": [],
        "line_flow": [],
    }
    n = net
    
    for line in n.lines:
        flow_out = line.p_flow_out_pu
        flow_in = line.p_flow_in_pu
        flow_max = line.flow_max_pu
        if abs(flow_out) > flow_max + tol or abs(flow_in) > flow_max + tol:
            violations["line_flow"].append((line.id, flow_out, flow_in, flow_max))
    for bus in n.buses:
        v_mag = bus.v_pu
        if v_mag < bus.v_min_pu - tol or v_mag > bus.v_max_pu + tol:
            violations["v_mag"].append((bus.id, v_mag, bus.v_min_pu, bus.v_max_pu))
    
    has_violations = any(len(v) > 0 for v in violations.values())
    return has_violations, violations

H = 24 # Horizonte de Simulação (horas)
NET = B3EOLCharged()  # Carrega sistema de teste
# C. Reseta RNGs e variáveis de estado
RNG1 = np.random.default_rng(seed=42)
RNG2 = np.random.default_rng(seed=41)
load_profile_base = np.array([
    0.70, 0.65, 0.62, 0.60, 0.65, 0.75, # 00:00 - 05:00 (Madrugada)
    0.85, 0.95, 1.00, 1.05, 1.10, 1.08, # 06:00 - 11:00 (Manhã/Almoço)
    1.05, 1.02, 1.00, 0.98, 1.05, 1.15, # 12:00 - 17:00 (Tarde)
    1.20, 1.18, 1.10, 1.00, 0.90, 0.80  # 18:00 - 23:00 (Noite/Pico)
])

# Perfil de Vento (Normalizado: 1.0 é a p_max_pu nominal)
# Vento costuma ser mais forte à noite e de madrugada, e cair durante o dia
wind_profile_base = np.array([
    0.90, 0.95, 0.98, 0.92, 0.85, 0.80, # Madrugada estável
    0.70, 0.60, 0.45, 0.30, 0.25, 0.35, # Queda matinal
    0.40, 0.30, 0.25, 0.35, 0.45, 0.55, # Tarde instável
    0.65, 0.75, 0.80, 0.85, 0.88, 0.92  # Recuperação noturna
])

generations_dc = []
generations_dc_corr = []
generations_ac = []
generations_nr = []
tensions_dc_corr = []
tensions_ac = []
tensions_nr = []
flows_dc = []
flows_dc_corr = []
flows_ac = []
flows_nr = []
fobs_dc = []
fobs_dc_corr = []
fobs_ac = []

net_sim_ac = deepcopy(NET)
net_sim_dc = deepcopy(NET)
net_sim_dc_corr = deepcopy(NET)
for h in range(H):
    # Solvers
    opf_dc = OPFIterativeLoss(net_sim_dc)
    opf_dc_corr = OPFIterativeLoss(net_sim_dc_corr)
    opf_ac = ACEconDispatch(net_sim_ac)

    # Aplica cenários de carga e vento direto, multiplicando perfis de NET e aplicando em cada rede que instanciamos

# 1. Aplica Carga nas 3 redes simultaneamente
    for l_base, l_dc, l_dc_corr, l_ac in zip(NET.loads, net_sim_dc.loads, net_sim_dc_corr.loads, net_sim_ac.loads):
        new_p = load_profile_base[h] * l_base.p_pu
        l_dc.p_pu = new_p
        l_dc_corr.p_pu = new_p
        l_ac.p_pu = new_p

    # 2. Aplica Eólica nas 3 redes simultaneamente
    for w_base, w_dc, w_dc_corr, w_ac in zip(NET.wind_generators, net_sim_dc.wind_generators, net_sim_dc_corr.wind_generators, net_sim_ac.wind_generators):
        new_wnd_max = wind_profile_base[h] * w_base.p_max_pu
        w_dc.p_max_pu = new_wnd_max
        w_dc_corr.p_max_pu = new_wnd_max
        w_ac.p_max_pu = new_wnd_max
    
    #Atualiza os solver que usam os parametros com mutable = True
    opf_dc.update_wind_params()
    opf_dc.update_load_params()
    opf_dc_corr.update_wind_params()
    opf_dc_corr.update_load_params()
    opf_ac.update_wind_params()
    opf_ac.update_load_params()

    # Resolve DC-OPF e AC-OPF
    results_dc_corr = opf_dc_corr.solve(verbose=False)
    results_dc = opf_dc.solve(verbose=False)
    results_ac = opf_ac.solve(verbose=False)

    #Atualiza a rede com os resultados
    opf_dc_corr.update_network_with_results()
    opf_dc.update_network_with_results()
    opf_ac.update_network_with_results()

    # Roda o Power Flow AC para checar violações na rede corrigida pelo DC-OPF
    power_flow = AC_PF(net_sim_dc_corr)
    power_flow.solve(verbose=False)

    # Atualiza a rede com os resultados do Power Flow
    power_flow.update_network_with_results()
    
    # Armazena resultados do Newton Raphson
    generations_nr.append([gen.p_mw for gen in net_sim_dc_corr.generators])
    tensions_nr.append([bus.v_pu for bus in net_sim_dc_corr.buses])
    flows_nr.append([max(branch.p_flow_out_pu, branch.p_flow_in_pu) for branch in net_sim_dc_corr.lines])

    # Checa violações
    has_violations, check_violationss = check_violations(net_sim_dc_corr)

    # Se houver violações, resolve o problema de minimização de desvios
    if has_violations:
        print(f"\n>>> Violations detected at hour {h}. Running AC Min Deviation OPF...")
        # Printa as violações antes e depois
        print("Violations:", check_violationss)
        min_deviation_opf = ACMinDeviation(net_sim_dc_corr)
        results_dc_corr = min_deviation_opf.solve(verbose=False)
        min_deviation_opf.update_network_with_results()
        has_violations_after, check_violationss_after = check_violations(net_sim_dc_corr)
        print("Violations after correction:", check_violationss_after)

    generations_dc.append([gen.p_mw for gen in net_sim_dc.generators])
    flows_dc.append([max(branch.p_flow_out_pu, branch.p_flow_in_pu) for branch in net_sim_dc.lines])
    fobs_dc.append(results_dc["Resumo"]["Total_Cost"])

    generations_dc_corr.append([gen.p_mw for gen in net_sim_dc_corr.generators])
    tensions_dc_corr.append([bus.v_pu for bus in net_sim_dc_corr.buses])
    flows_dc_corr.append([max(branch.p_flow_out_pu, branch.p_flow_in_pu) for branch in net_sim_dc_corr.lines])
    fobs_dc_corr.append(results_dc_corr["Resumo"]["Total_Cost"])

    generations_ac.append([gen.p_mw for gen in net_sim_ac.generators])
    tensions_ac.append([bus.v_pu for bus in net_sim_ac.buses])
    flows_ac.append([max(branch.p_flow_out_pu, branch.p_flow_in_pu) for branch in net_sim_ac.lines])
    fobs_ac.append(results_ac["Resumo"]["Total_Cost"])

    

Converged in 2 iterations.
Converged in 2 iterations.
Converged in 1 iterations.
Converged in 1 iterations.
Converged in 2 iterations.
Converged in 2 iterations.
Converged in 2 iterations.

>>> Violations detected at hour 6. Running AC Min Deviation OPF...
Violations: {'v_mag': [], 'line_flow': [(1, np.float64(0.020287685205850656), np.float64(-0.02024368068519332), 0.02), (3, np.float64(0.10051100738864913), np.float64(-0.09999735064506521), 0.1)]}
Violations after correction: {'v_mag': [], 'line_flow': []}
Converged in 2 iterations.

>>> Violations detected at hour 7. Running AC Min Deviation OPF...
Violations: {'v_mag': [], 'line_flow': [(1, np.float64(0.028617713043999987), np.float64(-0.028535645606919985), 0.02), (3, np.float64(0.10880297448089238), np.float64(-0.10820213322132832), 0.1)]}
Violations after correction: {'v_mag': [], 'line_flow': []}
Converged in 2 iterations.

>>> Violations detected at hour 8. Running AC Min Deviation OPF...
Violations: {'v_mag': [], 'line_flow':

In [4]:
# 1. Preparação dos Nomes e Dados
gen_names = [gen.id for gen in NET.generators]
horas = list(range(H))

def build_df(data, label):
    df = pd.DataFrame(data, columns=gen_names)
    df['Hora'] = horas
    df['Metodo'] = label
    return df

# Unificando as estratégias
df_all = pd.concat([
    build_df(generations_dc, "DC Puro"),
    build_df(generations_nr, "NR (Ponto DC)"),
    build_df(generations_dc_corr, "DC Corrigido"),
    build_df(generations_ac, "AC Full")
])

# 2. "Derretendo" os dados para o formato longo
df_melt = df_all.melt(id_vars=['Hora', 'Metodo'], var_name='Gerador', value_name='MW')

# 3. Plotagem em Plano Único
# Cor = Gerador | Estilo da Linha = Método
fig = px.line(
    df_melt, 
    x="Hora", 
    y="MW", 
    color="Gerador",      # Cada cor é um gerador (G1, G2, Wind...)
    line_dash="Metodo",   # Cada estilo de linha é um método (Puro, NR, Corrigido, AC)
    title="Comparativo Geral de Despacho: Todos os Geradores e Métodos",
    labels={"MW": "Potência Ativa (MW)", "Hora": "Hora do Dia"},
    template="plotly_white"
)

# Melhorando a interatividade
fig.update_layout(
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3, # Joga a legenda para baixo para não apertar o gráfico
        xanchor="center",
        x=0.5
    ),
    yaxis=dict(title="Geração (MW)"),
    xaxis=dict(tickmode='linear', tick0=0, dtick=1) # Garante que mostre todas as horas
)

fig.show()

In [5]:
import pandas as pd
import plotly.express as px

# 1. Escolha a hora do pico (ajuste conforme seu perfil de carga, ex: h=18)
hora_pico = 18

# 2. Organizando os dados para essa hora específica
bus_ids = [bus.id for bus in NET.buses]

# Criando um DataFrame comparativo para a hora selecionada
df_pico = pd.DataFrame({
    'Barra': bus_ids,
    'Newton-Raphson (DC)': tensions_nr[hora_pico],
    'DC Corrigido (MinDev)': tensions_dc_corr[hora_pico],
    'AC Full (Ótimo)': tensions_ac[hora_pico]
})

# Transformando para formato longo para o Plotly
df_pico_melt = df_pico.melt(id_vars='Barra', var_name='Método', value_name='Tensão (pu)')

# 3. Plotagem
fig_pico = px.line(
    df_pico_melt,
    x='Barra',
    y='Tensão (pu)',
    color='Método',
    markers=True,
    title=f'Perfil de Tensão por Barra - Hora de Pico (h={hora_pico})',
    template='plotly_white'
)

# Adicionando faixas de segurança
fig_pico.add_hline(y=0.94, line_dash="dash", line_color="red", annotation_text="Mín (0.94)")
fig_pico.add_hline(y=1.06, line_dash="dash", line_color="red", annotation_text="Máx (1.06)")

# Ajustes de layout
fig_pico.update_layout(
    xaxis_title="ID da Barra",
    yaxis_title="Tensão (pu)",
    hovermode="x unified",
    xaxis=dict(type='category') # Garante que as barras sejam tratadas como nomes
)

fig_pico.show()